In [3]:
## Import and authenticate
import ee
ee.Authenticate()
ee.Initialize()

Enter verification code: 4/1AX4XfWhOI6ZpYksoOcmr0vqAl_T2PX0xSbR4rx9SpBkxg31dnd8KGniu8EU

Successfully saved authorization token.


In [4]:
import pandas as pd

In [7]:
l8 = ee.ImageCollection("LANDSAT/LC08/C01/T1_SR")
geometry = ee.Geometry.Polygon([[[-80.92489760146748, 25.433457120928352],
                                 [-80.64474623427998, 25.488013471687964],
                                 [-80.57882826552998, 25.710940372707608],
                                 [-81.02377455459248, 25.770317250349557],
                                 [-80.95236342177998, 25.552457242621447]]])

#Filtering date, polygon, and cloudiness
images = (l8.filterDate('2013-09-01', '2021-12-31')
          .filterBounds(geometry)
          .filter('CLOUD_COVER < 5'))

#NDVI calculation:
def ndvi_func(i):
    ndvi = i.normalizedDifference (['B5', 'B4']).rename('NDVI')
    return i.addBands(ndvi)

images_ndvi = images.map(ndvi_func)

#Calculating year wise NDVI.  Result is a list of dictionaries.
years = ee.List.sequence(2013,2021)
def year_func(y):
    range = ee.Filter.calendarRange (y, y, 'year')
    mean = (images_ndvi
            .filter(range)
            .select('NDVI')
            .median()
            .reduceRegion(ee.Reducer.mean(), geometry, 30))
    return mean.set('year', y)

yearwise_ndvi = years.map(year_func).getInfo()
df = pd.DataFrame(yearwise_ndvi)
print(df)

   year      NDVI
0  2013       NaN
1  2014  0.381649
2  2015       NaN
3  2016  0.327974
4  2017  0.366617
5  2018  0.404349
6  2019  0.449728
7  2020       NaN
8  2021  0.430439
